In [1]:
import time
import pandas as pd
from datetime import datetime
import threading

import bernielib as bl

bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [2]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [3]:
ber.robotHome()
ber.pipetteHome()

# Preparing diluted stock solution

In [4]:
dna_stock = bl.createSample('eppendorf', 'dna_stock', ber.samples_rack, 0, 9, 120)
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 0)
water = bl.createSample('50ml', 'water', ber.reagents_rack, 1, 1, 35000)

In [5]:
script = pd.read_csv('mixing_pattern_eppendorf.csv')
ber.pickUpNextTip()
ber.move(z=50)
ber.transferLiquid(water, dna_stock_2, 980)
ber.transferLiquid(dna_stock, dna_stock_2, 20, v_immerse_dispense=1600)
ber.move(z=50)
ber.mixByScript(dna_stock_2, script)
ber.pipetteHome()
ber.move(z=50)
ber.dumpTipToWaste()

# Pre-loading samples with a ladder

In [6]:
initial_vol_list = [30, 30, 30, 30, 30, 30, 30, 30, 30]
samples_number = len(initial_vol_list)
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 1000)

In [7]:
ber.pickUpNextTip()
samples_list = bl.createSamplesToPurifyList(ber, [0 for x in range(samples_number)])

for sample, volume in zip(samples_list, initial_vol_list):
    ber.transferLiquid(dna_stock_2, sample, volume, v_immerse_dispense=1600, touch_wall=False)

ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()